# Chapitre 4.1 - Les fichiers CSV, JSON et les requêtes

## Le format CSV

Un fichier CSV est un fichier de tableau simplifié au maximum :
- le fichier est encodé en plein texte, il peut être lu par un éditeur de texte lambda
- le fichier utilise un séparateur de colonnes (usuellement une virgule, un point-virgule ou une tabulation)
- le fichier utilise une ligne par ligne de tableur

Exemple : [Les 1000 premiers numéros du Gaulois en statistiques](./data/csv/gaulois.csv) (grâce à la [BNF](http://api.bnf.fr/m%C3%A9tadonn%C3%A9es-quantitatives-de-la-presse-ancienne-xixe-xxe-si%C3%A8cles#chapitre3) )

## Les Packages

Python est fait de nombreuses fonctions de bases. Nous avons vu par exemple qu'il était possible d'utiliser facilement les fonctions `len()` ou encore `print()`. Mais Python possède aussi des `packages` (le nom des librairies et bibliothèques en Python) par défaut.

Un package est un ensemble de modules comportant des outils tels que des fonctions et qui peut être assez simplement importé. Par exemple, si je voulais travailler avec le package standard pour utiliser des fichiers CSV, je ferais :

In [18]:
import csv

Cette ligne va me permettre d'importer le module csv. Regardons ce qu'il a dans le ventre :

In [2]:
dir(csv)

['Dialect',
 'DictReader',
 'DictWriter',
 'Error',
 'OrderedDict',
 'QUOTE_ALL',
 'QUOTE_MINIMAL',
 'QUOTE_NONE',
 'QUOTE_NONNUMERIC',
 'Sniffer',
 'StringIO',
 '_Dialect',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 'excel',
 'excel_tab',
 'field_size_limit',
 'get_dialect',
 'list_dialects',
 're',
 'reader',
 'register_dialect',
 'unix_dialect',
 'unregister_dialect',
 'writer']

C'est bien, mais ça ne vous dit pas grand-chose non ? On voit à la limite quelque chose pour lire (`reader`) et quelque chose pour écrire (`writer`) avec *a priori* deux variations : `DictReader` et `DictWriter`. Mais cela reste vague. 

---

## Le package CSV

Heureusement, python propose pour ses librairies standards une très bonne documentation : https://docs.python.org/3.5/library/csv.html

Je reprends ci-dessous deux morceaux de la documentation, qui sont ceux qui vont d'abord nous intéresser.

### 1. Lire

Commençons par ce qui est la documentation de `reader()`
 
#### csv.reader(csvfile, dialect=’excel’, **fmtparams)

> Return a reader object which will iterate over lines in the given `csvfile`. `csvfile` can be any object which supports the iterator protocol and returns a string each time its `__next__()` method is called — file objects and list objects are both suitable. If csvfile is a file object, it should be opened with newline=''. [1] An optional `dialect` parameter can be given which is used to define a set of parameters specific to a particular CSV dialect. It may be an instance of a subclass of the Dialect class or one of the strings returned by the list_dialects() function. The other optional `fmtparams` keyword arguments can be given to override individual formatting parameters in the current dialect. For full details about the dialect and formatting parameters, see section [Dialects and Formatting Parameters](https://docs.python.org/3.5/library/csv.html#dialects-and-formatting-parameters).

> Each row read from the csv file is returned as a list of strings. No automatic data type conversion is performed unless the QUOTE_NONNUMERIC format option is specified (in which case unquoted fields are transformed into floats).

> A short usage example:

In [3]:
import csv
with open('data/csv/eggs.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(row)
        print(', '.join(row))
# Spam, Spam, Spam, Spam, Spam, Baked Beans
# Spam, Lovely Spam, Wonderful Spam

['Spam', 'Spam', 'Spam', 'Spam', 'Spam', 'Baked Beans']
Spam, Spam, Spam, Spam, Spam, Baked Beans
['Spam', 'Lovely Spam', 'Wonderful Spam']
Spam, Lovely Spam, Wonderful Spam


**À partir de cette documentation et de cet exemple**, on peut tirer plusieurs conclusions :

- `csv.reader()` prend comme premier argument un fichier ouvert
- il peut prendre un `dialect` ou des paramètres qui ne sont pas développés par la documentation de la fonction elle-même : dans `**fmtparams`, `**` signifie qu'il existe d'autres paramètres optionnels nominatifs.
- il arrive que, sans aller dans *Dialects and Formatting Parameters*, nous avons deux exemples de ces paramètres dans l'exemple :
    - `delimiter` qui semble être un délimiteur de colonne.
    - `quotechar` qui semble être un "encapsulateur" permettant d'échapper les délimiteurs donc délimite les chaînes de caractères
    
Regardons le fichier [data/csv/eggs.csv](data/csv/eggs.csv)

En lançant l'exemple, on s'aperçoit que le `reader` va renvoyer une liste quand on va itérer dessus: ces listes correspondent aux lignes.

###### Une fonction utile : la fonction enumerate

Dans le cadre d'une boucle, enumerate permet de renvoyer un tuple sur une valeur simple afin de de compter l'index de l'objet parcouru :

In [6]:
couleurs = ["vert", "rouge", "bleu"]

for index, couleur in enumerate(couleurs):
    print(str(index)+ " = " + couleur)

#revient au même que
index = 0
for couleur in couleurs:
    index += 1
    print(str(index - 1)+ " = " + couleur)

0 = vert
1 = rouge
2 = bleu
0 = vert
1 = rouge
2 = bleu


##### Exercice 

À partir du fichier `data/csv/gaulois.csv`, compter le nombre de publicités en page 1 sur l'ensemble des numéros. 

*Attention ! La première ligne est une ligne d'en-tête*

In [10]:
import csv
pubs = 0
with open('data/csv/gaulois.csv') as csvfile:
    pubreader = csv.reader(csvfile) #attention, quotechar n'est que pour les chaînes de caractères
    #permet de sauter la première ligne
    next(pubreader)
    for row in pubreader:
        pubs += int(row[9])        

# Tests pour vérifier : le résultat doit être enregistré dans une variable pubs
#attention aux types des choses de la liste!
assert pubs == 2260, "Il y a eu 2260 publicité sur les premiers numéros du Gaulois"

### 2. Écrire

Si lire est utile, écrire l'est tout autant, surtout lorsque l'on récupère des informations d'ici et là et que l'on veut les remettre en formes

Voyons la documentation de csv.writer et son exemple :

#### csv.writer(csvfile, dialect=’excel’, **fmtparams)

>    Return a writer object responsible for converting the user’s data into delimited strings on the given file-like object. csvfile can be any object with a write() method. If csvfile is a file object, it should be opened with newline='' [1]. An optional dialect parameter can be given which is used to define a set of parameters specific to a particular CSV dialect. It may be an instance of a subclass of the Dialect class or one of the strings returned by the list_dialects() function. The other optional fmtparams keyword arguments can be given to override individual formatting parameters in the current dialect. For full details about the dialect and formatting parameters, see section Dialects and Formatting Parameters. To make it as easy as possible to interface with modules which implement the DB API, the value None is written as the empty string. While this isn’t a reversible transformation, it makes it easier to dump SQL NULL data values to CSV files without preprocessing the data returned from a cursor.fetch* call. All other non-string data are stringified with str() before being written.

> A short usage example:

In [9]:
import csv
with open('data/csv/eggs.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['Spam'] * 5 + ['Baked Beans'])
    spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])

In [14]:
import csv
with open("resultats/exemple_csv.csv", "w") as f:
    csvwriter = csv.writer(f, delimiter = "\t")
    csvwriter.writerow(["titre", "note"])
    csvwriter.writerow(["blanche_neige", "7"])

In [17]:
import csv
with open("resultats/exemple_csv.csv", "r") as f:
    csvwriter = csv.reader(f, delimiter = "\t")
    for row in csvwriter:
        print(row)

['titre', 'note']
['blanche_neige', '7']


**À partir de cette documentation et de cet exemple**, on peut tirer plusieurs conclusions :

- `csv.writer()` prend comme premier argument un fichier ouvert en mode écriture
- il peut prendre un `dialect` ou des paramètres qui ne sont pas développés par la documentation de la fonction elle-même : dans `**fmtparams`, `**` signifie qu'il existe d'autres paramètres optionnels nominatifs.
- il arrive que, sans aller dans *Dialects and Formatting Parameters*, nous ayons deux exemples de ces paramètres dans l'exemple :
    - `delimiter` qui semble être un délimiteur de colonne.
    - `quotechar` qui semble être un "encapsulateur" permettant d'échapper les délimiteurs
    - `quoting` qui d'après la documentation correspond à un mode de citation minimal (Utilisation des `quotechar` que lorsque cela est nécessaire. C'est par ailleurs la valeur par défaut : https://docs.python.org/3.5/library/csv.html#csv.Dialect.quoting )
- on écrit une ligne en utilisant la méthode `.writerow()` qui prend comme argument une liste
- on utilise `.writerow()` autant de fois que nécessaire
    
Regardons le fichier [data/csv/eggs.csv](data/csv/eggs.csv)

En lançant l'exemple, on s'aperçoit que le `writer()` va rédiger deux lignes. Ce sont les deux que nous avons lues plus haut.

##### Exercice

`modules_cours.chapitre4` est un module écrit spécifiquement pour ce cours (si vous êtes [curieux-ses](modules_cours/chapitre4.py)) Dans ce module, il y a la fonction `read_rss()` qui renvoie un générateur contenant des tuples Titre - Résumé - Lien - Date de publication. Par exemple :

In [2]:
from modules_cours.chapitre4 import read_rss

# Il s'agit ici de l'adresse RSS d'un flux de la BNF: flux RSS c'est liste des derniers 
# articles publiés: sous la forme de XML avec tags prévus dans espace de nom donnés avec dates, noms, etc
for entree in read_rss("http://www.bibliopat.fr/rss-actualites"):
    print(entree)

('Journées BiblioPat 2019 - Le patrimoine MOUVEMENTé', '<div class="field field-name-body field-type-text-with-summary field-label-hidden"><div class="field-items"><div class="field-item even" property="content:encoded"><div class="field field-name-body field-type-text-with-summary field-label-hidden">\n<div class="field-items">\n<div class="field-item even">\n<p>Les journées d\'étude BiblioPat 2019\xa0<em>Le patrimoine MOUVEMENTé</em> ont lieu ces deux jours ci (7 et 8 novembre 2019) dans l\'amphithéâtre de l\'Académie de Médecine (Paris). Suivez les discussions en direct sur Twitter sur le compte de <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"><a href="https://twitter.com/BiblioPat_fr">@BiblioPat_fr</a> et avec le mot-dièse </span><span class="r-18u37iz"><a class="css-4rbku5 css-18t94o4 css-901oao css-16my406 r-1n1174f r-1loqt21 r-1qd0xha r-ad9z0x r-bcqeeo r-1ny4l3l r-1ddef8g r-qvutc0 ext" data-focusable="true" dir="ltr" href="https://twitter.com/hashtag/

À partir de cette fonction, réaliser un CSV (en utilisant le délimiteur `\t`) pour le flux RSS de Bibliopat : `http://www.bibliopat.fr/rss-actualites`. Le CSV sera stocké à `bibliopat.csv`.

In [9]:
from modules_cours.chapitre4 import read_rss
import csv

# Votre code ici

with open("resultats/bibliopat.csv", "w") as f:
    for entree in read_rss("http://www.bibliopat.fr/rss-actualites"):
        csvwriter = csv.writer(f, delimiter = "\t")
        csvwriter.writerow([entree[0],entree[1],entree[2],entree[3]])

##### Avez-vous remarqué ?

Ci-dessus, nous avons utilisé `from ___ import ___`. Cette structure d'import nous permet de n'importer qu'un sous-module ou une sous-fonction en particulier. Très pratique pour éviter les noms à rallonge !

----

#### Ce que l'on a appris

Pour finir cette section, voici un récapitulatif des concepts appris. Lisez la liste et posez des questions si certaines choses ne sont pas claires.

- `import`
- `from ___ import ___`
- la notion de librairie/module/package
- `csv.reader()`
- `csv.writer()`
- `enumerate()`
- Lire une documentation et ses exemples

#### Ce que l'on recommande d'approfondir

Si les fonctions `csv.reader()` et `csv.writer()` sont sympathiques, les fonctions `DictReader` et `DictWriter` le sont beaucoup plus : elles stockent les noms de colonnes ! *cf.* [la documentation](https://docs.python.org/3.5/library/csv.html#csv.DictReader)